In [1]:
from utils import *

df = load_and_sample_data()
if df is not None:
    print(f"Loaded DataFrame with shape: {df.shape}")
    print(df.head())
else:
    print("Failed to load data.")

 Found cached sample at: ..\data\ml-20m\ratings_cleaned_sampled.csv
Loaded DataFrame with shape: (1000000, 3)
   userId  movieId  rating
0   49400     4011       5
1  118685      786       3
2   80510     1247       3
3  124963     1721       1
4   31031     1982       4
